# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 8:50AM,256668,10,0086355391,ISDIN Corporation,Released
1,Feb 6 2023 8:50AM,256668,10,0086355406,ISDIN Corporation,Released
2,Feb 6 2023 8:50AM,256668,10,0086355896,ISDIN Corporation,Released
3,Feb 6 2023 8:50AM,256668,10,0086355918,ISDIN Corporation,Released
4,Feb 6 2023 8:50AM,256668,10,0086355980,ISDIN Corporation,Released
5,Feb 6 2023 8:50AM,256668,10,0086356430,ISDIN Corporation,Released
6,Feb 6 2023 8:50AM,256668,10,0086358959,ISDIN Corporation,Released
7,Feb 6 2023 8:46AM,256667,12,U1208,"Uniderm USA, Inc",Released
8,Feb 6 2023 8:45AM,256666,10,SONSB0002098,"Nextsource Biotechnology, LLC",Released
9,Feb 6 2023 8:44AM,256665,10,EYE43022,Eye Pharma Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,256664,Released,1
9,256665,Released,1
10,256666,Released,1
11,256667,Released,1
12,256668,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256664,NaN,1.0
256665,NaN,1.0
256666,NaN,1.0
256667,NaN,1.0
256668,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256655,0.0,2.0
256656,0.0,3.0
256657,0.0,1.0
256659,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256655,0,2
256656,0,3
256657,0,1
256659,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256655,0,2
2,256656,0,3
3,256657,0,1
4,256659,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256655,,2
2,256656,,3
3,256657,,1
4,256659,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation
7,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc"
8,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC"
9,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc
10,Feb 6 2023 8:43AM,256663,12,Hush Hush
67,Feb 6 2023 8:39AM,256664,12,Hush Hush
68,Feb 6 2023 8:38AM,256662,16,ECI Pharmaceuticals LLC
69,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC"
70,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc."
71,Feb 6 2023 8:11AM,256657,20,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,,7
1,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",,1
2,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC",,1
3,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc,,1
4,Feb 6 2023 8:43AM,256663,12,Hush Hush,,57
5,Feb 6 2023 8:39AM,256664,12,Hush Hush,,1
6,Feb 6 2023 8:38AM,256662,16,ECI Pharmaceuticals LLC,,1
7,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",,1
8,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",,1
9,Feb 6 2023 8:11AM,256657,20,Else Nutrition,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,7,
1,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",1,
2,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC",1,
3,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc,1,
4,Feb 6 2023 8:43AM,256663,12,Hush Hush,57,
5,Feb 6 2023 8:39AM,256664,12,Hush Hush,1,
6,Feb 6 2023 8:38AM,256662,16,ECI Pharmaceuticals LLC,1,
7,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",1,
8,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",1,
9,Feb 6 2023 8:11AM,256657,20,Else Nutrition,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,7,
1,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",1,
2,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC",1,
3,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc,1,
4,Feb 6 2023 8:43AM,256663,12,Hush Hush,57,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,7.0,NaN
1,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",1.0,NaN
2,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC",1.0,NaN
3,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc,1.0,NaN
4,Feb 6 2023 8:43AM,256663,12,Hush Hush,57.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,7.0,0.0
1,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",1.0,0.0
2,Feb 6 2023 8:45AM,256666,10,"Nextsource Biotechnology, LLC",1.0,0.0
3,Feb 6 2023 8:44AM,256665,10,Eye Pharma Inc,1.0,0.0
4,Feb 6 2023 8:43AM,256663,12,Hush Hush,57.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,769999,9.0,0.0
12,769994,59.0,0.0
16,513323,2.0,0.0
19,769970,6.0,0.0
20,256657,1.0,0.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,769999,9.0,0.0
1,12,769994,59.0,0.0
2,16,513323,2.0,0.0
3,19,769970,6.0,0.0
4,20,256657,1.0,0.0
5,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,9.0,0.0
1,12,59.0,0.0
2,16,2.0,0.0
3,19,6.0,0.0
4,20,1.0,0.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,12,Released,59.0
2,16,Released,2.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,22
Status,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,1.0
Released,9.0,59.0,2.0,6.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,22
0,Executing,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,9.0,59.0,2.0,6.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,22
0,Executing,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,9.0,59.0,2.0,6.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()